In [6]:
import sqlparse

from cast.cache import PostData

from django.test.client import RequestFactory
from django.db import connection, reset_queries

In [7]:
def show_queries(queries):
    for query in queries:
        formatted_sql = sqlparse.format(query['sql'], reindent=True, keyword_case='upper')
        print(formatted_sql)
        

def blocker(*args):
    raise Exception("No database access allowed here.")

In [8]:
# blog_slug = "ephes_blog"
blog_slug = "das_claas_und_nora_blog"
blog = Blog.objects.get(slug=blog_slug)

In [9]:
%%time
reset_queries()
post_queryset = Post.objects.live().descendant_of(blog).order_by("-visible_date")
request_factory = RequestFactory()
post_data = PostData.create_from_post_queryset(
    request=request_factory.get("/"),
    blog=blog,
    post_queryset=post_queryset,
    template_base_dir="bootstrap4",
)

CPU times: user 399 ms, sys: 44.2 ms, total: 444 ms
Wall time: 468 ms


In [15]:
%%time
request = request_factory.get("/foobar/")
posts = list(post_queryset)
reset_queries()
# request = request_factory.get(post_data.page_url_by_id[post.pk])
for post in posts:
    # print("post pk: ", post.pk)
    with connection.execute_wrapper(blocker):
        post._local_template_name = "post_body.html"
        rendered_content = post.serve(
            request, template_base_dir="bootstrap4", post_data=post_data
        ).rendered_content
    # rendered_content = post.serve(
    #     request, template_base_dir="bootstrap4"
    # ).rendered_content        
    # response = post.serve(request, post_data=post_data).render()
    # description = post.get_description(
    #     request=request, escape_html=False, remove_newlines=False,
    #     render_detail=True, post_data=post_data
    # )

CPU times: user 868 ms, sys: 66.8 ms, total: 935 ms
Wall time: 938 ms


In [14]:
print(len(connection.queries))

3558


In [12]:
connection.queries

[]

# Make Sure the Feed Works Without Database

In [13]:
from cast.feeds import LatestEntriesFeed

In [17]:
feed = LatestEntriesFeed()
feed.post_data = post_data

PostData(renditions_for_posts=1810, template_base_dir=bootstrap4)